In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
basePath ='./sims.TNG/TNG50-1/output'

from FunctionLib import *
from tqdm import tqdm
current_snap_num=99
import gc

In [2]:
subhalos_data=il.groupcat.loadSubhalos(basePath,current_snap_num,fields=['SubhaloMass','SubhaloPos'])

In [3]:
subhalo_mass=subhalos_data['SubhaloMass']
subhalo_pos=subhalos_data['SubhaloPos']
subhalo_pos=np.mod(subhalo_pos,35000)

In [7]:
condition=((subhalo_mass>10) & (subhalo_mass<5000))
index_list_0=np.where(condition)[0]
index_list_0.shape

(1447,)

In [5]:
np.where(sorted_index_array==329508)

(array([61]),)

In [10]:
stars_mass_list=[]
for subhalo_index in tqdm(index_list_0):
    stars_mass=il.snapshot.loadSubhalo(basePath,current_snap_num,subhalo_index,'stars',fields=['Masses'])
    if (stars_mass.sum() < 1):continue
    stars_mass_list.append(subhalo_index)
index_array=np.array(stars_mass_list)
index_array.shape

100%|██████████| 1447/1447 [01:49<00:00, 13.19it/s]


(633,)

In [11]:
R_90_list=[]
selected_index_list=[]
for subhalo_index in tqdm(index_array):
    
    stars_mass=il.snapshot.loadSubhalo(basePath,current_snap_num,subhalo_index,'stars',fields=['Masses'])
    if (stars_mass.sum() < 1):continue
    gas_data=il.snapshot.loadSubhalo(basePath,current_snap_num,subhalo_index,'gas',fields=['Coordinates','Masses','NeutralHydrogenAbundance'])
    if len(gas_data)==1:continue
    
    gas_mass=gas_data['Masses']
    gas_HI_abundance=gas_data['NeutralHydrogenAbundance']
    gas_HI_mass=gas_mass*gas_HI_abundance
    gas_coordinates=gas_data['Coordinates']
    gas_coordinates=np.mod(gas_coordinates,35000)
    radius=1000
    
    R_90=calculate_R90(0,radius,gas_HI_mass,gas_coordinates,subhalo_pos[subhalo_index])
    
    R_90_list.append(R_90)
    selected_index_list.append(subhalo_index)
    del gas_data,gas_mass,gas_HI_abundance,gas_HI_mass,gas_coordinates,R_90,radius
    
    

100%|██████████| 633/633 [14:32<00:00,  1.38s/it]


In [12]:
selected_index_array=np.array(selected_index_list)
R_90_array=np.array(R_90_list)

In [15]:
tuples_list = list(zip(selected_index_array, R_90_array))
sorted_tuples = sorted(tuples_list, key=lambda x: x[1], reverse=True)
sorted_index_array = [t[0] for t in sorted_tuples]
sorted_R_90_array = [t[1] for t in sorted_tuples]

In [6]:
np.save('sorted_index_array.npy',sorted_index_array)
np.save('R_90_array.npy',sorted_R_90_array)

In [5]:
sorted_index_array=np.load('sorted_index_array.npy')
sorted_R_90_array=np.load('R_90_array.npy')

In [ ]:
plt.hist(sorted_R_90_array,bins=200)

In [4]:
def drawsubhalo(i,sp,savepath,basePath):
    index=np.where(sorted_index_array==i)

    gas_data=il.snapshot.loadSubhalo(basePath, sp, i, 'gas', fields=['Coordinates','NeutralHydrogenAbundance','Masses'])
    if len(gas_data)==1:return -1
    
    stars_mass=il.snapshot.loadSubhalo(basePath,current_snap_num,i,'stars',fields=['Masses'])
    
    cold_gas_mass =Cold_Gas_Mass(sp,i)
    
    GasCoordinates = gas_data['Coordinates']
    GasCoordinates=np.mod(GasCoordinates,35000)
    GasAbundance = gas_data['NeutralHydrogenAbundance']
    Masses = gas_data['Masses']
    NeutralHydrogenAbundance=Masses*GasAbundance
    
    GasCoorMin = GasCoordinates.min(axis=0)
    GasCoorMax = GasCoordinates.max(axis=0)
    bars=2*(GasCoorMax-GasCoorMin)
    if (bars[0]>8000)or (bars[2]>8000):return -1
    plt.figure(figsize=(20, 20), dpi=400)

    plt.style.use("dark_background")

    h, _, _, image = plt.hist2d(GasCoordinates[:, 0],
                            GasCoordinates[:, 2],
                            weights=NeutralHydrogenAbundance,
                            
                            norm=mpl.colors.LogNorm(),
                            bins=(bars[0],bars[2]))
    plt.xlim([GasCoorMin[0],GasCoorMax[0]])
    plt.ylim([GasCoorMin[2],GasCoorMax[2]])
    plt.xlabel('x [ckpc/h]')
    plt.ylabel('z [ckpc/h]')
    plt.text(0.65*(GasCoorMax[0]-GasCoorMin[0])+GasCoorMin[0],0.82*(GasCoorMax[2]-GasCoorMin[2])+GasCoorMin[2],'HI_R_90={:.2f}kpc\nM_Stars={:.2f}E10Ms\nM_Galaxy={:.2f}E10Ms\nM_Gas={:.2f}E10Ms\nM_CG={:.2f}E10Ms\nM_HI={:.2f}E10Ms'.format(sorted_R_90_array[np.where(sorted_index_array==i)[0]][0],stars_mass.sum(),subhalo_mass[i],Masses.sum(),cold_gas_mass,NeutralHydrogenAbundance.sum()),color='white',fontsize=22)

    draw_circle = plt.Circle((subhalo_pos[i][0], subhalo_pos[i][2]),sorted_R_90_array[index][0] , fill=False)
    plt.gcf().gca().add_artist(draw_circle)
    plt.colorbar(image)
    plt.gca().set_aspect(1)
    plt.savefig(savepath+'/{}.png'.format(i))
    plt.close('all')
    plt.clf()
    del index, GasCoorMin,GasCoorMax,gas_data,stars_mass,cold_gas_mass,GasCoordinates,GasAbundance,Masses,NeutralHydrogenAbundance,bars,image,draw_circle
    gc.collect()
    return 0

In [10]:
drawsubhalo(329508,99,'GalaxiesHIxz',basePath)

0

<Figure size 432x288 with 0 Axes>

In [11]:
for i in tqdm(sorted_index_array[58:100]):
    drawsubhalo(i,99,'GalaxiesHIxz',basePath)

100%|██████████| 42/42 [15:58<00:00, 22.82s/it]


<Figure size 432x288 with 0 Axes>

In [6]:
def Cold_Gas_Mass(snap_num,subhalo_id):
    gas_fields=['ElectronAbundance','InternalEnergy','Masses']
    subhalo_data=il.snapshot.loadSubhalo(basePath,snap_num,subhalo_id,0,fields=gas_fields)
    if len(subhalo_data)==1:
        return 0
    
    x_e=subhalo_data['ElectronAbundance'].astype(np.float64)
    internal_energy=subhalo_data['InternalEnergy'].astype(np.float64)
    gas_cell_masses=subhalo_data['Masses'].astype(np.float64)
    
    m_p=1.673E-24
    X_H=0.76
    unit_switching=1E10
    mean_molecular_weight=4*m_p/(1+3*X_H+4*X_H*x_e)
    k_B=1.38E-16
    gas_cell_temperature_in_Kelvin=2/3*internal_energy/k_B*unit_switching*mean_molecular_weight
    cold_gas_mass=0
    
    for temperature,gas_cell_mass in Binary_DataIter(gas_cell_temperature_in_Kelvin,gas_cell_masses):
        if temperature<10000:cold_gas_mass+=gas_cell_mass    
    del subhalo_data,x_e,internal_energy,gas_cell_masses,mean_molecular_weight,gas_cell_temperature_in_Kelvin
    gc.collect()
    return cold_gas_mass

In [10]:
gc.collect()

0

In [8]:
del subhalos_data